In [1]:
from mindspore import context
from mindspore.nn import Adam
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import DynamicUpdater
from mindsponge import SimulationCell
from mindsponge.control import VelocityVerlet, Langevin
from mindsponge.potential import SphericalRestrict
from mindsponge.function import VelocityGenerator
from mindsponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
mini = Sponge(system, potential, opt)

In [8]:
run_info = RunInfo(10)
mini.run(1000, callbacks=[run_info])

Step: 0, E_pot: -9490.139, 
Step: 10, E_pot: -10437.1875, 
Step: 20, E_pot: -10485.595, 
Step: 30, E_pot: -10503.794, 
Step: 40, E_pot: -10539.535, 
Step: 50, E_pot: -10562.064, 
Step: 60, E_pot: -10585.358, 
Step: 70, E_pot: -10605.756, 
Step: 80, E_pot: -10624.354, 
Step: 90, E_pot: -10642.369, 
Step: 100, E_pot: -10660.057, 
Step: 110, E_pot: -10676.273, 
Step: 120, E_pot: -10690.239, 
Step: 130, E_pot: -10702.393, 
Step: 140, E_pot: -10713.615, 
Step: 150, E_pot: -10724.471, 
Step: 160, E_pot: -10734.634, 
Step: 170, E_pot: -10743.717, 
Step: 180, E_pot: -10751.804, 
Step: 190, E_pot: -10759.179, 
Step: 200, E_pot: -10766.087, 
Step: 210, E_pot: -10772.567, 
Step: 220, E_pot: -10778.411, 
Step: 230, E_pot: -10783.281, 
Step: 240, E_pot: -10787.031, 
Step: 250, E_pot: -10789.912, 
Step: 260, E_pot: -10792.288, 
Step: 270, E_pot: -10794.341, 
Step: 280, E_pot: -10796.144, 
Step: 290, E_pot: -10797.754, 
Step: 300, E_pot: -10799.222, 
Step: 310, E_pot: -10800.573, 
Step: 320, E_pot: -

In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.coordinate.shape, system.atom_mass)

In [10]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, temp),
                     velocity=velocity,
                     time_step=5e-4)

## 外加限制

在MindSponge的SimulationCell中可以自定义一些外加的势能项，这是一个非常灵活的操作，支持同时定义多个Bias叠加使用，并且在MindSpore框架的支持下都是可微的。比如本案例中施加的Bias是中心球形势SphericalRestrict，用于把原子的运动限制在一个球形范围内。另外MindSponge也支持原位谐振子势OscillatorBias，可以将原子限制在原始位置附近，这个功能在蛋白质预测中能够起到较大的作用。基于MindSponge的基础类PotentialCell，用户还可以自定义更多种类的Bias。

In [11]:
sim = SimulationCell(system, potential, bias=SphericalRestrict(radius=1.5, center=[0, 0, 0]))
md = Sponge(sim, optimizer=opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b04.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.run(3000, callbacks=[run_info, cb_h5md])

Step: 0, E_pot: -10841.856, E_kin: 142.05127, E_tot: -10699.805, Temperature: 325.42557
Step: 10, E_pot: -10789.327, E_kin: 85.47205, E_tot: -10703.855, Temperature: 195.8081
Step: 20, E_pot: -10771.497, E_kin: 81.908356, E_tot: -10689.589, Temperature: 187.64403
Step: 30, E_pot: -10794.112, E_kin: 91.955246, E_tot: -10702.157, Temperature: 210.66046
Step: 40, E_pot: -10780.373, E_kin: 77.26148, E_tot: -10703.111, Temperature: 176.99849
Step: 50, E_pot: -10785.424, E_kin: 90.77876, E_tot: -10694.6455, Temperature: 207.96526
Step: 60, E_pot: -10790.228, E_kin: 93.40996, E_tot: -10696.817, Temperature: 213.99306
Step: 70, E_pot: -10774.354, E_kin: 73.70509, E_tot: -10700.648, Temperature: 168.85115
Step: 80, E_pot: -10775.463, E_kin: 76.50136, E_tot: -10698.962, Temperature: 175.25711
Step: 90, E_pot: -10783.381, E_kin: 83.34915, E_tot: -10700.031, Temperature: 190.94473
Step: 100, E_pot: -10795.066, E_kin: 92.320015, E_tot: -10702.746, Temperature: 211.49611
Step: 110, E_pot: -10780.713